In [1]:
# Import data and packages
import pandas as pd, numpy as np
import json

from os import listdir
from os.path import isfile, join

import scispacy, spacy
nlp = spacy.load('en_core_sci_md')

In [2]:
fileDirectory1 = '/home/rahul/DeepBlue/Projects/CORD-19/noncomm_use_subset/noncomm_use_subset'
fileDirectory2 = '/home/rahul/DeepBlue/Projects/CORD-19/comm_use_subset/comm_use_subset'
fileDirectory3 = '/home/rahul/DeepBlue/Projects/CORD-19/biorxiv_medrxiv/biorxiv_medrxiv'
fileDirectory4 = '/home/rahul/DeepBlue/Projects/CORD-19/custom_license/custom_license'
jsonFiles1 = [join(fileDirectory1,f) for f in listdir(fileDirectory1) if isfile(join(fileDirectory1,f))]
jsonFiles2 = [join(fileDirectory2,f) for f in listdir(fileDirectory2) if isfile(join(fileDirectory2,f))]
jsonFiles3 = [join(fileDirectory3,f) for f in listdir(fileDirectory3) if isfile(join(fileDirectory3,f))]
jsonFiles4 = [join(fileDirectory4,f) for f in listdir(fileDirectory4) if isfile(join(fileDirectory4,f))]
jsonFiles = jsonFiles1+jsonFiles2+jsonFiles3+jsonFiles4
print(f'Number of files: {len(jsonFiles)}')

Number of files: 29315


In [ ]:
from tqdm.notebook import tqdm
dictList = []
for f in tqdm(jsonFiles):
    with open(f) as currFile:
        data = json.load(currFile)
    dictList.append(data)
df = pd.DataFrame(dictList)
df.head()

In [ ]:
titles = []
for i in range(len(df)):
    titles.append(df.metadata[i]['title'])
df2 = pd.DataFrame()
df2['paper_id'] = df['paper_id']
df2['title'] = titles
df2.to_csv('PaperID_Title.csv', index=False)
df2.head()

In [5]:
df.metadata[0]['authors']

[{'first': 'Luke',
  'middle': ['D'],
  'last': 'Knibbs',
  'suffix': '',
  'affiliation': {'laboratory': 'International Laboratory for Air Quality and Health',
   'institution': 'Queensland University of Technology',
   'location': {'settlement': 'Brisbane',
    'region': 'Queensland',
    'country': 'Australia'}},
  'email': ''},
 {'first': 'Graham',
  'middle': ['R'],
  'last': 'Johnson',
  'suffix': '',
  'affiliation': {'laboratory': 'International Laboratory for Air Quality and Health',
   'institution': 'Queensland University of Technology',
   'location': {'settlement': 'Brisbane',
    'region': 'Queensland',
    'country': 'Australia'}},
  'email': ''},
 {'first': 'Timothy',
  'middle': ['J'],
  'last': 'Kidd',
  'suffix': '',
  'affiliation': {'laboratory': '',
   'institution': 'The University of Queensland',
   'location': {'settlement': 'Herston',
    'region': 'Queensland',
    'country': 'Australia'}},
  'email': ''},
 {'first': 'Joyce',
  'middle': [],
  'last': 'Cheney

In [6]:
def RemovePunctuation(text):
#    import string
#    table = str.maketrans("","", string.punctuation)
#    return text.translate(table)
    text = text.replace('-',' ')
    return_text = [c for c in text if (c.isalpha() or c==' ')]
    return_text = ''.join(return_text)
    return return_text

In [7]:
print(RemovePunctuation('aaa.bg!s'))

aaabgs


In [8]:
def RemoveCitationsFromParagraph(df, doc_id, para_id):
    remove_indices = []
    for i in range(len(df['body_text'][doc_id][para_id]['cite_spans'])):
        curr_start = df['body_text'][doc_id][para_id]['cite_spans'][i]['start']
        curr_end = df['body_text'][doc_id][para_id]['cite_spans'][i]['end']
        for k in range(curr_start,curr_end):
            remove_indices.append(k)
    remove_indices = set(remove_indices)
    text_return = ''.join([char for idx,char in enumerate(df['body_text'][doc_id][para_id]['text'])
                           if idx not in remove_indices]) 
    return text_return

In [9]:
def GetPaperAuthors(df, doc_id):
    try:
        authors = df.metadata[doc_id]['authors'][0]['last']
        for i in range(1,len(df.metadata[doc_id]['authors'])):
            authors = authors + ', ' + df.metadata[doc_id]['authors'][i]['last']
    except:
        authors='NA'
    return authors

In [10]:
def ConvertToLemma(text):
    doc = nlp(text)
    return [word.lemma_ for word in doc if not(word.like_num or word.is_stop or word.is_punct or word.is_space)]

In [11]:
import re
from tqdm.notebook import tqdm

doc_id = []
doc_title = []
doc_txt = []
doc_authors = []
for i in tqdm(range(len(df))):
    doc_id.append(df.paper_id[i])
    doc_title.append(df.metadata[i]['title'])
    doc_authors.append(GetPaperAuthors(df,i))
    temp = []
    for k in range(len(df['body_text'][i])):
        curr_para = RemoveCitationsFromParagraph(df,i,k)
        temp.append(curr_para)
    curr_txt = ' '.join(temp) # This text has no citations in it
    curr_txt = re.sub(r'http\S+', '', curr_txt) # This text has no websites in it
    curr_txt = RemovePunctuation(curr_txt) # This text has no punctuation in it
    curr_txt = curr_txt.lower() # This text is all in lower case
    #curr_txt = ConvertToLemma(curr_txt) # This text has word lemmas instead of full words
    doc_txt.append(curr_txt)
df2 = pd.DataFrame()
df2['doc_id'] = doc_id
df2['title'] = doc_title
df2['txt'] = doc_txt
df2['author'] = doc_authors
df2.head()

,doc_id,title,txt,author
0,df1b6713549bcbe22ec020aaf62275280f096b9a,Viability of Pseudomonas aeruginosa in cough a...,communicable respiratory infections are an imp...,"Knibbs, Johnson, Kidd, Cheney, Grimwood, Katte..."
1,84a328211fa0a65959af45f0e55e52961c00005f,Oxidized LDL upregulates macrophage DPP4 expre...,dipeptidyl peptidase iv dpp is a single pass t...,"Rao, Zhao, Braunstein, Mao, Razavi, Duan, Wei,..."
2,817a885e9613363e08ef920a9e5826a4cb9e1c1e,CLINICAL EXPERIMENTAL VACCINE RESEARCH,virulent ndv the causative agent of newcastle ...,NA
3,3de90c8c18dbed90408c4a5cc5cb94f856c9f637,,in humans cyclosporiasis and cystoisosporiasis...,"Lee, Kim, Cheon, Moon, Seo, Jung, Shin, Choi"
4,14e878a6400d8320e2c781a76edc4a14f4cec1df,Viral etiology of acute respiratory infections...,introduction acute respiratory tract infection...,"Masoud, Hanna-Wakim, Zaraket, Araj, Matar, Dbaibo"


In [ ]:
# Create a lemma column where all words are lemmatised
from tqdm.notebook import tqdm

# Increase max length of document NLP will look at
nlp.max_length = 1e7
unique_lemmas = []
lemmas = []
for i in tqdm(range(min(1000,len(df2)))):
    doc = nlp(df2.txt[i])
    if(len(doc)>nlp.max_length): # Clip documents to max length
        doc = doc[0:nlp.max_length]
    doclemma = []
    uniquedoclemma = []
    for token in doc:
        doclemma.append(token.lemma_)
        if(token.lemma_ not in uniquedoclemma):
            uniquedoclemma.append(token.lemma_)
    lemmas.append(doclemma)
    unique_lemmas.append(uniquedoclemma)
df2['lemmas'] = lemmas
df2['unique_lemmas'] = unique_lemmas
df2.to_pickle("DFWithLemmas_part1.pkl")

In [ ]:
# Create a lemma column where all words are lemmatised
from tqdm.notebook import tqdm

# Increase max length of document NLP will look at
nlp.max_length = 1e7
lemmas = []
for i in tqdm(range(len(df2))):
    doc = nlp(df2.txt[i])
    if(len(doc)>nlp.max_length): # Clip documents to max length
        doc = doc[0:nlp.max_length]
    doclemma = []
    for token in doc:
        doclemma.append(token.lemma_)
    lemmas.append(doclemma)
df2['nonunique_lemmas'] = lemmas